In [9]:
#set konfigurasi googledrive
from google.colab import drive

In [10]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
data_file = '/content/drive/My Drive/tugasnlp/spamfilter/SMSSpamCollection.txt'

#load dataset
import pandas as pd

data = pd.read_csv(data_file, sep='\t', header=None, names=["label", "sms"])
data.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [23]:
#firststep
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(stopwords[:5])
print(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['i', 'me', 'my', 'myself', 'we']
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [24]:
def pre_process(sms):
    remove_punct = "".join([word.lower() for word in sms if word not in punctuation])
    tokenize = nltk.tokenize.word_tokenize(remove_punct)
    remove_stopwords = [word for word in tokenize if word not in stopwords]
    return remove_stopwords

#tambahkan kolom ke data pesan yang mau diproses
data['processed'] = data['sms'].apply(lambda x: pre_process(x))

print(data['processed'].head())

0    [go, jurong, point, crazy, available, bugis, n...
1                       [ok, lar, joking, wif, u, oni]
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3        [u, dun, say, early, hor, u, c, already, say]
4    [nah, dont, think, goes, usf, lives, around, t...
Name: processed, dtype: object


In [25]:
#mengelompokkan dan hitung token, menentukan threshold

def categorize_words():
    spam_words = []
    ham_words = []
    #memproses pesan yang di golongkan spam
    for sms in data['processed'][data['label'] == 'spam']:
        for word in sms:
            spam_words.append(word)
    #memproses pesan yang di golongkan ham
    for sms in data['processed'][data['label'] == 'ham']:
        for word in sms:
            ham_words.append(word)
    return spam_words, ham_words
spam_words, ham_words = categorize_words()

print(spam_words[:5])
print(ham_words[:5])

['free', 'entry', '2', 'wkly', 'comp']
['go', 'jurong', 'point', 'crazy', 'available']


In [29]:
#memprediksi fungsi dengan threshold 5:5 x 100 = {}%
def predict(sms):
    spam_counter = 0
    ham_counter = 0
    #mengakumulasikan kemunculkan kata sms
    for word in sms:
        spam_counter += spam_words.count(word)
        ham_counter += ham_words.count(word)
    print('**HASIL AKHIR**')
    #pesan spam dikalikan 100
    if ham_counter > spam_counter:
        accuracy = round((ham_counter / (ham_counter + spam_counter) * 100))
        print('pesan tidak mengandung spam, angka keputusan {}%'.format(accuracy))
    #menampilkan pesan ham atau spam
    elif ham_counter == spam_counter:
        print('pesan mengandung spam')
    #menampilkan pesan ham atau spam berdasarkan akurasinya
    else :
        accuracy = round((ham_counter / (ham_counter + spam_counter) * 100))
        print('pesan mengandung spam, angka keputusan {}%'.format(accuracy))

In [46]:
user_input = input("Inputkan spam atau ham guna memeriksa keakuratan fungsi\n")

Inputkan spam atau ham guna memeriksa keakuratan fungsi
Free trial in 1 mnth comp to use Nrt accnt,please txt to 900900d


In [47]:
processed_input = pre_process(user_input)
predict(processed_input)

**HASIL AKHIR**
pesan mengandung spam, angka keputusan 28%
